In [1]:
# different imports
import pandas as pd
import numpy as np
import pickle
import os

Note: change preproseeing to new pipeline

In [2]:
# Import data that has already been preprocessed
data_dir = os.getcwd()
with open(os.path.join(data_dir, "processing", r"train_preproc.p"), 'rb') as data_file:
    train_data = pickle.load(data_file)
X_train, y_train = train_data[0], train_data[1]

with open(os.path.join(data_dir, "processing", r"test_preproc.p"), 'rb') as data_file:
    test_data = pickle.load(data_file)
X_test, y_test = test_data[0], test_data[1]



In [3]:
# Test that there are no nans in train data
for column in X_train.columns:
    assert sum(X_train[column].isnull()) == 0

In [ ]:
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier

param_grid = {
    'alpha': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e-0],
    'penalty': ['l1','l2', 'elasticnet'],
    'n_jobs': [-1],
    'loss' : ['perceptron', 'squared_loss', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive', 'modified_huber','log', 'squared_hinge' ]
}


gs = GridSearchCV(SGDClassifier(random_state=42),
                  param_grid, refit='AUC', return_train_score=True)
gs.fit(X_train, y_train)
results = gs.cv_results_

print("The best parameters are %s with a score of %0.2f"
      % (gs.best_params_, gs.best_score_))


In [ ]:
# For finding the best parameters first RandomizedSearchCV (to narrow down the parameters space) 
# then GridSearch For finding the best parameters

from sklearn.datasets import load_iris
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

iris = load_iris()
sgd = SGDClassifier(tol=1e-3, max_iter=10000, random_state=0, n_jobs=-1)
distributions = dict(alpha=uniform(loc=0, scale=1), penalty=['l2', 'l1','elasticnet'],loss = ['perceptron', 'squared_loss', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive', 'modified_huber','log', 'squared_hinge'] )

clf = RandomizedSearchCV(sgd, distributions, refit='roc_auc')#find best params based on AUC
search = clf.fit(iris.data, iris.target)
search.best_params_


In [ ]:
results = clf.cv_results_
#results

In [ ]:
from sklearn.linear_model import SGDClassifier
classifier = SGDClassifier(loss='log', alpha=0.04374782771527885, max_iter=10000, penalty ='l1', n_jobs=-1) # parameters to be adjusted to make most optimal 


SKEvaluator should work with SGD since RF class has implicit functions fit, predict, 
predict_proba which are used inside SKEvaluator--> need to set loss=”modified_huber” though to have internal predict_prba call

In [ ]:
#from ModelEvaluationTools.SKEvaluator import SKEvaluator
#sgd_model = SKEvaluator(classifier, "sgd")
#sgd_model.fit(X_train, y_train)

In [ ]:
#test_auc = sgd_model.evaluate(X_test, y_test) #evaluate here seems to be using predict_proba(X_test) but i need to use decision_function(X_test) with some loss functions

In [ ]:
#play around with auc explicitly
from sklearn.metrics import roc_auc_score
classifier.fit(X_train, y_train)

In [ ]:
roc_auc_score(y_test, classifier.decision_function(X_test))

In [ ]:
# can only use this if loss=”modified_huber”
roc_auc_score(y_test, classifier.predict_proba(X_test)[:, 1])

In [ ]:
#trying to find the best parameters for the classifier
from sklearn.model_selection import ParameterGrid
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import roc_auc_score
import parfit.parfit as pf
grid = {
    'alpha': [1e-4, 1e-3, 1e-2, 1e-1],
    'penalty': ['l1'],
    'n_jobs': [-1],
    'loss' : ['perceptron', 'squared_loss', 'huber', 'modified_huber','log', 'squared_hinge' ]
}
#'perceptron', 'squared_loss', 'huber', 
#'epsilon_insensitive', 'squared_epsilon_insensitive'
# 'modified_huber','log', 'squared_hinge'
paramGrid = ParameterGrid(grid)
bestModel, bestScore, allModels, allScores = (
pf.bestFit(SGDClassifier, paramGrid,
           X_train, y_train, X_test, y_test, 
           metric = roc_auc_score, 
           scoreLabel = "AUC"))
#print(bestModel, bestScore)

Best found so far With parfit: SGDClassifier(loss='huber', n_jobs=-1, penalty='l1') 0.5100757967187792

In [ ]:
# saves model to fitted_models folder
#sgd_model.save_model()